In [ ]:
# default_exp inference

# Inference

> This contains the code required for inference.

In [ ]:
# export
from fastai.learner import load_learner
from fastai.callback.core import GatherPredsCallback
from fastai.learner import Learner
from fastcore.basics import patch
from fastcore.meta import delegates

In [ ]:
#export
@patch
@delegates(GatherPredsCallback.__init__)
def get_X_preds(self: Learner, X, y=None, bs=64, with_input=False, with_decoded=True, with_loss=False, **kwargs):
    if with_loss and y is None:
        print('cannot find loss as y=None')
        with_loss = False
    dl = self.dls.valid.new_dl(X, y=y)
    dl.bs = self.dls.bs if bs is None else bs
    output = list(self.get_preds(dl=dl, with_input=with_input, with_decoded=with_decoded, with_loss=with_loss, reorder=False))
    if with_decoded and hasattr(self.dls, 'vocab'):
        output[2 + with_input] = L([self.dls.vocab[p] for p in output[2 + with_input]])
    return tuple(output)

In [ ]:
from tsai.data.external import get_UCR_data
dsid = 'OliveOil'
X, y, splits = get_UCR_data(dsid, split_data=False)
X_test = X[splits[1]]
y_test = y[splits[1]]

In [ ]:
learn = load_learner("./models/test.pth")

In [ ]:
test_probas, test_targets, test_preds = learn.get_X_preds(X_test[0:10], with_decoded=True)
test_probas, test_targets, test_preds

(tensor([[0.1761, 0.1498, 0.1514, 0.5227],
         [0.1762, 0.1498, 0.1514, 0.5226],
         [0.1761, 0.1498, 0.1513, 0.5228],
         [0.1761, 0.1498, 0.1514, 0.5227],
         [0.1761, 0.1498, 0.1513, 0.5228],
         [0.1762, 0.1498, 0.1514, 0.5225],
         [0.1762, 0.1498, 0.1514, 0.5225],
         [0.1762, 0.1498, 0.1514, 0.5226],
         [0.1762, 0.1499, 0.1514, 0.5225],
         [0.1762, 0.1498, 0.1515, 0.5225]]),
 None,
 tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3]))

In [ ]:
test_probas2, test_targets2, test_preds2 = learn.get_X_preds(X_test[0:10], y_test[0:10], with_decoded=True)
test_probas2, test_targets2, test_preds2

(tensor([[0.1761, 0.1498, 0.1514, 0.5227],
         [0.1762, 0.1498, 0.1514, 0.5226],
         [0.1761, 0.1498, 0.1513, 0.5228],
         [0.1761, 0.1498, 0.1514, 0.5227],
         [0.1761, 0.1498, 0.1513, 0.5228],
         [0.1762, 0.1498, 0.1514, 0.5225],
         [0.1762, 0.1498, 0.1514, 0.5225],
         [0.1762, 0.1498, 0.1514, 0.5226],
         [0.1762, 0.1499, 0.1514, 0.5225],
         [0.1762, 0.1498, 0.1515, 0.5225]]),
 TensorCategory([0, 0, 0, 0, 0, 1, 1, 1, 1, 1]),
 tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3]))

In [ ]:
from fastcore.test import test_eq
test_eq(test_probas, test_probas2)
test_eq(test_preds, test_preds2)

In [ ]:
#hide
from tsai.imports import create_scripts
from tsai.export import get_nb_name
nb_name = get_nb_name()
create_scripts(nb_name);

<IPython.core.display.Javascript object>

052a_inference.ipynb saved at 2021-11-22 23:37:43.
Converted 052a_inference.ipynb.


Correct conversion! 😃
Total time elapsed 0.095 s
Monday 22/11/21 23:37:46 CET
